In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from numpy import array
from sklearn.preprocessing import LabelEncoder

In [2]:
#read the data

data = pd.read_excel('../bestsellers with categories.xlsx')

In [3]:
data.describe()

,User Rating,Reviews,Price,Year
count,550.000000,550.000000,550.000000,550.000000
mean,4.618364,11953.281818,13.100000,2014.000000
std,0.226980,11731.132017,10.842262,3.165156
min,3.300000,37.000000,0.000000,2009.000000
25%,4.500000,4058.000000,7.000000,2011.000000
50%,4.700000,8580.000000,11.000000,2014.000000
75%,4.800000,17253.250000,16.000000,2017.000000
max,4.900000,87841.000000,105.000000,2019.000000


In [4]:
data

,Name,Author,User Rating,Reviews,Price,Year,Genre
0,10-Day Green Smoothie Cleanse,JJ Smith,4.7,17350,8,2016,Non Fiction
1,11/22/63: A Novel,Stephen King,4.6,2052,22,2011,Fiction
2,12 Rules for Life: An Antidote to Chaos,Jordan B. Peterson,4.7,18979,15,2018,Non Fiction
3,1984 (Signet Classics),George Orwell,4.7,21424,6,2017,Fiction
4,"5,000 Awesome Facts (About Everything!) (Natio...",National Geographic Kids,4.8,7665,12,2019,Non Fiction
...,...,...,...,...,...,...,...
545,Wrecking Ball (Diary of a Wimpy Kid Book 14),Jeff Kinney,4.9,9413,8,2019,Fiction
546,You Are a Badass: How to Stop Doubting Your Gr...,Jen Sincero,4.7,14331,8,2016,Non Fiction
547,You Are a Badass: How to Stop Doubting Your Gr...,Jen Sincero,4.7,14331,8,2017,Non Fiction
548,You Are a Badass: How to Stop Doubting Your Gr...,Jen Sincero,4.7,14331,8,2018,Non Fiction


In [5]:
jk = data[(data['Author'] == 'Jeff Kinney') & (data['Price'] != 0)]
jk_ave_price = jk['Price'].mean()

sm = data[(data['Author'] == 'Stephenie Meyer') & (data['Price'] != 0)]
sm_ave_price = sm['Price'].mean()

hl = data[(data['Author'] == 'Harper Lee') & (data['Price'] != 0)]
hl_ave_price = hl['Price'].mean()

rh = data[(data['Author'] == 'RH Disney') & (data['Price'] != 0)]
rh_ave_price = rh['Price'].mean()

ac = data[(data['Author'] == 'Alice Schertle') & (data['Price'] != 0)]
ac_ave_price = ac['Price'].mean()

consti = data[(data['Name'] == 'The Constitution of the United States') & (data['Price'] != 0)]
consti_ave_price = consti['Price'].mean()

In [39]:
data.loc[(data['Price'] == 0) & (data['Author'] == 'Jeff Kinney'), 'Price'] = jk_ave_price
data.loc[(data['Price'] == 0) & (data['Author'] == 'Stephenie Meyer'), 'Price'] = sm_ave_price
data.loc[(data['Price'] == 0) & (data['Author'] == 'Harper Lee'), 'Price'] = hl_ave_price
data.loc[(data['Price'] == 0) & (data['Author'] == 'RH Disney'), 'Price'] = rh_ave_price
data.loc[(data['Price'] == 0) & (data['Author'] == 'Alice Schertle'), 'Price'] = ac_ave_price
data.loc[(data['Price'] == 0) & (data['Name'] == 'The Constitution of the United States'), 'Price'] = consti_ave_price


print(data[data['Author'] == 'RH Disney'])
print(data[data['Author'] == 'Alice Schertle'])
print(data[data['Name'] == 'The Constitution of the United States'])


#drop NAN values
data = data.dropna()
data = data.drop("level_0", axis=1)


Empty DataFrame
Columns: [level_0, Name, Author, User Rating, Reviews, Price, Year, Genre]
Index: []
Empty DataFrame
Columns: [level_0, Name, Author, User Rating, Reviews, Price, Year, Genre]
Index: []
Empty DataFrame
Columns: [level_0, Name, Author, User Rating, Reviews, Price, Year, Genre]
Index: []


In [40]:
data[data['Price'] == 0]
data.info()
data = data.reset_index()
data = data.drop("index", axis=1)
data

<class 'pandas.core.frame.DataFrame'>
Int64Index: 546 entries, 0 to 545
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         546 non-null    object 
 1   Author       546 non-null    object 
 2   User Rating  546 non-null    float64
 3   Reviews      546 non-null    int64  
 4   Price        546 non-null    float64
 5   Year         546 non-null    int64  
 6   Genre        546 non-null    object 
dtypes: float64(2), int64(2), object(3)
memory usage: 34.1+ KB


,Name,Author,User Rating,Reviews,Price,Year,Genre
0,10-Day Green Smoothie Cleanse,JJ Smith,4.7,17350,8.0,2016,Non Fiction
1,11/22/63: A Novel,Stephen King,4.6,2052,22.0,2011,Fiction
2,12 Rules for Life: An Antidote to Chaos,Jordan B. Peterson,4.7,18979,15.0,2018,Non Fiction
3,1984 (Signet Classics),George Orwell,4.7,21424,6.0,2017,Fiction
4,"5,000 Awesome Facts (About Everything!) (Natio...",National Geographic Kids,4.8,7665,12.0,2019,Non Fiction
...,...,...,...,...,...,...,...
541,Wrecking Ball (Diary of a Wimpy Kid Book 14),Jeff Kinney,4.9,9413,8.0,2019,Fiction
542,You Are a Badass: How to Stop Doubting Your Gr...,Jen Sincero,4.7,14331,8.0,2016,Non Fiction
543,You Are a Badass: How to Stop Doubting Your Gr...,Jen Sincero,4.7,14331,8.0,2017,Non Fiction
544,You Are a Badass: How to Stop Doubting Your Gr...,Jen Sincero,4.7,14331,8.0,2018,Non Fiction


In [41]:

counters = {}
dropnames = []
names = data['Name']
for i in range(len(names)):
    if names[i] in counters:
        counters[names[i]] += 1
        dropnames.append(i)
    else:
        counters[names[i]] = 1
data_dropped = data.drop(dropnames)
data_dropped['Occurences'] = counters.values()
data_dropped = data_dropped.reset_index()
data_dropped = data_dropped.drop("index", axis=1)
data_dropped

,Name,Author,User Rating,Reviews,Price,Year,Genre,Occurences
0,10-Day Green Smoothie Cleanse,JJ Smith,4.7,17350,8.0,2016,Non Fiction,1
1,11/22/63: A Novel,Stephen King,4.6,2052,22.0,2011,Fiction,1
2,12 Rules for Life: An Antidote to Chaos,Jordan B. Peterson,4.7,18979,15.0,2018,Non Fiction,1
3,1984 (Signet Classics),George Orwell,4.7,21424,6.0,2017,Fiction,1
4,"5,000 Awesome Facts (About Everything!) (Natio...",National Geographic Kids,4.8,7665,12.0,2019,Non Fiction,1
...,...,...,...,...,...,...,...,...
342,Winter of the World: Book Two of the Century T...,Ken Follett,4.5,10760,15.0,2012,Fiction,1
343,Women Food and God: An Unexpected Path to Almo...,Geneen Roth,4.2,1302,11.0,2010,Non Fiction,1
344,Wonder,R. J. Palacio,4.8,21625,9.0,2013,Fiction,5
345,Wrecking Ball (Diary of a Wimpy Kid Book 14),Jeff Kinney,4.9,9413,8.0,2019,Fiction,1


In [42]:
authors = data_dropped['Author']

In [43]:
stop_words = nltk.corpus.stopwords.words('english')
stop_words
def clean_text(document):
    tokenizer = RegexpTokenizer('\w+')
    pst = PorterStemmer()
    words = tokenizer.tokenize(document)

    words = [pst.stem(w.lower()) for w in words if w.lower() not in stop_words]
    sentence = ' '.join(words)
    return sentence

In [44]:
clean_names_title = []
uniquenames = names.unique()
for i in range(len(uniquenames)):
    clean = clean_text(uniquenames[i])
    clean_names_title.append(clean)
clean_names_title = np.array(clean_names_title)
data_dropped['Clean Name'] = clean_names_title
vec = TfidfVectorizer()
cv = vec.fit_transform(data_dropped['Clean Name']).toarray()
data_dropped['IDF Vector'] = vec.fit_transform(data_dropped['Clean Name'])
data_dropped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 347 entries, 0 to 346
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         347 non-null    object 
 1   Author       347 non-null    object 
 2   User Rating  347 non-null    float64
 3   Reviews      347 non-null    int64  
 4   Price        347 non-null    float64
 5   Year         347 non-null    int64  
 6   Genre        347 non-null    object 
 7   Occurences   347 non-null    int32  
 8   Clean Name   347 non-null    object 
 9   IDF Vector   347 non-null    object 
dtypes: float64(2), int32(1), int64(2), object(5)
memory usage: 25.9+ KB


In [45]:
clean_names_author = []
for i in range(len(authors)):
    clean = clean_text(authors[i])
    clean_names_author.append(clean)
clean_names_author = np.array(clean_names_author)
data_dropped['Clean Name Author'] = clean_names_author
cv = vec.fit_transform(data_dropped['Clean Name Author']).toarray()
data_dropped['IDF Vector Authors'] = vec.fit_transform(data_dropped['Clean Name Author'])
data_dropped

,Name,Author,User Rating,Reviews,Price,Year,Genre,Occurences,Clean Name,IDF Vector,Clean Name Author,IDF Vector Authors
0,10-Day Green Smoothie Cleanse,JJ Smith,4.7,17350,8.0,2016,Non Fiction,1,10 day green smoothi cleans,"(0, 164)\t0.4852828294287546\n (0, 830)\t0....",jj smith,"(0, 370)\t0.6481914304908\n (0, 186)\t0.761..."
1,11/22/63: A Novel,Stephen King,4.6,2052,22.0,2011,Fiction,1,11 22 63 novel,"(0, 164)\t0.4852828294287546\n (0, 830)\t0....",stephen king,"(0, 370)\t0.6481914304908\n (0, 186)\t0.761..."
2,12 Rules for Life: An Antidote to Chaos,Jordan B. Peterson,4.7,18979,15.0,2018,Non Fiction,1,12 rule life antidot chao,"(0, 164)\t0.4852828294287546\n (0, 830)\t0....",jordan b peterson,"(0, 370)\t0.6481914304908\n (0, 186)\t0.761..."
3,1984 (Signet Classics),George Orwell,4.7,21424,6.0,2017,Fiction,1,1984 signet classic,"(0, 164)\t0.4852828294287546\n (0, 830)\t0....",georg orwel,"(0, 370)\t0.6481914304908\n (0, 186)\t0.761..."
4,"5,000 Awesome Facts (About Everything!) (Natio...",National Geographic Kids,4.8,7665,12.0,2019,Non Fiction,1,5 000 awesom fact everyth nation geograph kid,"(0, 164)\t0.4852828294287546\n (0, 830)\t0....",nation geograph kid,"(0, 370)\t0.6481914304908\n (0, 186)\t0.761..."
...,...,...,...,...,...,...,...,...,...,...,...,...
342,Winter of the World: Book Two of the Century T...,Ken Follett,4.5,10760,15.0,2012,Fiction,1,winter world book two centuri trilog,"(0, 164)\t0.4852828294287546\n (0, 830)\t0....",ken follett,"(0, 370)\t0.6481914304908\n (0, 186)\t0.761..."
343,Women Food and God: An Unexpected Path to Almo...,Geneen Roth,4.2,1302,11.0,2010,Non Fiction,1,women food god unexpect path almost everyth,"(0, 164)\t0.4852828294287546\n (0, 830)\t0....",geneen roth,"(0, 370)\t0.6481914304908\n (0, 186)\t0.761..."
344,Wonder,R. J. Palacio,4.8,21625,9.0,2013,Fiction,5,wonder,"(0, 164)\t0.4852828294287546\n (0, 830)\t0....",r j palacio,"(0, 370)\t0.6481914304908\n (0, 186)\t0.761..."
345,Wrecking Ball (Diary of a Wimpy Kid Book 14),Jeff Kinney,4.9,9413,8.0,2019,Fiction,1,wreck ball diari wimpi kid book 14,"(0, 164)\t0.4852828294287546\n (0, 830)\t0....",jeff kinney,"(0, 370)\t0.6481914304908\n (0, 186)\t0.761..."


In [46]:
genres = data_dropped['Genre']
values = array(genres)
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
data_dropped['Genre'] = integer_encoded
data_dropped['IDF Vector']
data_dropped

,Name,Author,User Rating,Reviews,Price,Year,Genre,Occurences,Clean Name,IDF Vector,Clean Name Author,IDF Vector Authors
0,10-Day Green Smoothie Cleanse,JJ Smith,4.7,17350,8.0,2016,1,1,10 day green smoothi cleans,"(0, 164)\t0.4852828294287546\n (0, 830)\t0....",jj smith,"(0, 370)\t0.6481914304908\n (0, 186)\t0.761..."
1,11/22/63: A Novel,Stephen King,4.6,2052,22.0,2011,0,1,11 22 63 novel,"(0, 164)\t0.4852828294287546\n (0, 830)\t0....",stephen king,"(0, 370)\t0.6481914304908\n (0, 186)\t0.761..."
2,12 Rules for Life: An Antidote to Chaos,Jordan B. Peterson,4.7,18979,15.0,2018,1,1,12 rule life antidot chao,"(0, 164)\t0.4852828294287546\n (0, 830)\t0....",jordan b peterson,"(0, 370)\t0.6481914304908\n (0, 186)\t0.761..."
3,1984 (Signet Classics),George Orwell,4.7,21424,6.0,2017,0,1,1984 signet classic,"(0, 164)\t0.4852828294287546\n (0, 830)\t0....",georg orwel,"(0, 370)\t0.6481914304908\n (0, 186)\t0.761..."
4,"5,000 Awesome Facts (About Everything!) (Natio...",National Geographic Kids,4.8,7665,12.0,2019,1,1,5 000 awesom fact everyth nation geograph kid,"(0, 164)\t0.4852828294287546\n (0, 830)\t0....",nation geograph kid,"(0, 370)\t0.6481914304908\n (0, 186)\t0.761..."
...,...,...,...,...,...,...,...,...,...,...,...,...
342,Winter of the World: Book Two of the Century T...,Ken Follett,4.5,10760,15.0,2012,0,1,winter world book two centuri trilog,"(0, 164)\t0.4852828294287546\n (0, 830)\t0....",ken follett,"(0, 370)\t0.6481914304908\n (0, 186)\t0.761..."
343,Women Food and God: An Unexpected Path to Almo...,Geneen Roth,4.2,1302,11.0,2010,1,1,women food god unexpect path almost everyth,"(0, 164)\t0.4852828294287546\n (0, 830)\t0....",geneen roth,"(0, 370)\t0.6481914304908\n (0, 186)\t0.761..."
344,Wonder,R. J. Palacio,4.8,21625,9.0,2013,0,5,wonder,"(0, 164)\t0.4852828294287546\n (0, 830)\t0....",r j palacio,"(0, 370)\t0.6481914304908\n (0, 186)\t0.761..."
345,Wrecking Ball (Diary of a Wimpy Kid Book 14),Jeff Kinney,4.9,9413,8.0,2019,0,1,wreck ball diari wimpi kid book 14,"(0, 164)\t0.4852828294287546\n (0, 830)\t0....",jeff kinney,"(0, 370)\t0.6481914304908\n (0, 186)\t0.761..."


In [47]:
data_nonalpha = data_dropped.drop(['Name', 'Author', 'Clean Name Author', 'Clean Name', 'IDF Vector Authors', 'IDF Vector'], axis=1)
scaler = StandardScaler()
data_nonalpha
data_scaled = scaler.fit_transform(data_nonalpha)
data_scaled_df = pd.DataFrame(data_scaled)
data_scaled_df.columns = ['User Rating', 'Reviews', 'Price', 'Year', 'Genre', 'Occurences']
data_scaled_df['IDF Vector Titles'] = data_dropped['IDF Vector']
data_scaled_df['IDF Vector Authors'] = data_dropped['IDF Vector Authors']
data_scaled_df

,User Rating,Reviews,Price,Year,Genre,Occurences,IDF Vector Titles,IDF Vector Authors
0,0.409117,0.685546,-0.550427,0.757679,0.909019,-0.449481,"(0, 164)\t0.4852828294287546\n (0, 830)\t0....","(0, 370)\t0.6481914304908\n (0, 186)\t0.761..."
1,-0.031764,-0.719512,0.866498,-0.760304,-1.100087,-0.449481,"(0, 164)\t0.4852828294287546\n (0, 830)\t0....","(0, 370)\t0.6481914304908\n (0, 186)\t0.761..."
2,0.409117,0.835162,0.158036,1.364873,0.909019,-0.449481,"(0, 164)\t0.4852828294287546\n (0, 830)\t0....","(0, 370)\t0.6481914304908\n (0, 186)\t0.761..."
3,0.409117,1.059726,-0.752845,1.061276,-1.100087,-0.449481,"(0, 164)\t0.4852828294287546\n (0, 830)\t0....","(0, 370)\t0.6481914304908\n (0, 186)\t0.761..."
4,0.849998,-0.203981,-0.145591,1.668469,0.909019,-0.449481,"(0, 164)\t0.4852828294287546\n (0, 830)\t0....","(0, 370)\t0.6481914304908\n (0, 186)\t0.761..."
...,...,...,...,...,...,...,...,...
342,-0.472645,0.080281,0.158036,-0.456707,-1.100087,-0.449481,"(0, 164)\t0.4852828294287546\n (0, 830)\t0....","(0, 370)\t0.6481914304908\n (0, 186)\t0.761..."
343,-1.795288,-0.788397,-0.246800,-1.063901,0.909019,-0.449481,"(0, 164)\t0.4852828294287546\n (0, 830)\t0....","(0, 370)\t0.6481914304908\n (0, 186)\t0.761..."
344,0.849998,1.078187,-0.449218,-0.153111,-1.100087,2.685595,"(0, 164)\t0.4852828294287546\n (0, 830)\t0....","(0, 370)\t0.6481914304908\n (0, 186)\t0.761..."
345,1.290879,-0.043435,-0.550427,1.668469,-1.100087,-0.449481,"(0, 164)\t0.4852828294287546\n (0, 830)\t0....","(0, 370)\t0.6481914304908\n (0, 186)\t0.761..."


In [48]:
data_scaled_df.to_csv('preprocessdf.csv')